In [8]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from math import ceil, floor, sqrt

In [9]:
class CUBE(object):
  def __init__(self, fname):
    f = open(fname, 'r')
    for i in range(2): f.readline() # echo comment
    tkns = f.readline().split() # number of atoms included in the file followed by the position of the origin of the volumetric data
    self.natoms = int(tkns[0])
    self.origin = np.array([float(tkns[1]),float(tkns[2]),float(tkns[3])])
    tkns = f.readline().split() #
    self.NX = int(tkns[0])
    self.X = np.array([float(tkns[1]),float(tkns[2]),float(tkns[3])])
    tkns = f.readline().split() #
    self.NY = int(tkns[0])
    self.Y = np.array([float(tkns[1]),float(tkns[2]),float(tkns[3])])
    tkns = f.readline().split() #
    self.NZ = int(tkns[0])
    self.Z = np.array([float(tkns[1]),float(tkns[2]),float(tkns[3])])
    self.atoms = []
    for i in range(self.natoms):
      tkns = f.readline().split()
      self.atoms.append([tkns[0], tkns[2], tkns[3], tkns[4]])
    self.data = np.zeros((self.NX,self.NY,self.NZ))
    i=0
    for s in f:
      for v in s.split():
        self.data[i//(self.NY*self.NZ), (i//self.NZ)%self.NY, i%self.NZ] = float(v)
        i+=1
    if i != self.NX*self.NY*self.NZ: raise NameError("FSCK!")


In [21]:
def get_rhotilde(boxpath):
    rdensities = {}
    rlvals = []
    
    # read
    for folder in glob.glob('%s/lval-*/' % boxpath):
        basename = folder.strip('/').split('/')[-1]
        mix = float(basename.split('-')[-1])
        rlvals.append(float(mix))
        cube = CUBE('%s/DENSITY.cube' % (folder))
        rdensities[mix] = cube.data
    if 0.0 not in rlvals or 1.0 not in rlvals:
        raise ValueError('Endpoints missing')
    
    # sort
    lvals = sorted(rlvals)
    densities = [rdensities[_] for _ in lvals]
       
    dviews = [_.reshape(-1) for _ in densities]
    rhotilde = np.trapz(np.array(dviews).T, np.array(lvals)).reshape(densities[0].shape)
    return cube.NX, cube.NY, cube.NZ, cube.X, cube.Y, cube.Z, rhotilde
def get_energies_cube(coords, zs, rhotilde, NX, NY, NZ, X, Y, Z):
    energies = np.zeros(len(zs))
    for x in np.arange(NX):
        for y in np.arange(NY):
            for z in np.arange(NZ):
                ds = np.linalg.norm(np.array([x*X[0],y*Y[1],z*Z[2]]) - coords, axis=1)
                for site in range(len(zs)):
                    energies[site] += zs[site] * rhotilde[x, y, z] / ds[site]
    return energies*X[0]*Y[1]*Z[2]
def discover_calculations(basepath):
    return glob.glob('%s/*/box-*/' % basepath)
lookup = {1: 'H', 6: 'C', 7: 'N', 8: 'O'}
rlookup = {'C': 6, 'O': 8, 'N': 7, 'B': 5, 'H': 1}
def read_xyz(fn):
    lines = open(fn).readlines()[2:]
    elements = []
    coords = []
    for line in lines:
        parts = line.strip().split()
        elements.append(rlookup[parts[0]])
        coords.append([float(_) for _ in parts[1:]])
    return elements, coords

In [22]:
def get_energies(boxpath):
    NX, NY, NZ, X, Y, Z, rhotilde = get_rhotilde(boxpath)
    zs, coords = read_xyz(boxpath + '../inp.xyz')
    return get_energies_cube(coords, zs, rhotilde, NX, NY, NZ, X, Y, Z)

In [29]:
def build_db():
    for boxpath in discover_calculations('/mnt/c/Users/guido/workcopies/apdft/prototyping/cpmd-uniform-electron-gas/data'):
        if 'NN' not in boxpath:
            continue
        try:
            energies = get_energies(boxpath)
        except:
            raise
            continue
        print (boxpath, energies)
build_db()

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Users/guido/workcopies/apdft/prototyping/cpmd-uniform-electron-gas/data/NN-2/box-12/lval-0.000000//DENSITY.cube'